In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
from keras.callbacks import ModelCheckpoint


with io.open("Tweets_no-toxic_pre_all.txt", encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


corpus length: 140845
total chars: 28


In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

#Model architecture
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

nb sequences: 46935
Vectorization...
Build model...


In [ ]:
verbose = 1
def train_model(model, X, y, batch_size=128, nb_epoch=100, verbose=0):
    checkpointer = ModelCheckpoint(filepath="weights_E.hdf5", monitor='loss', verbose=verbose, save_best_only=True, mode='min')
    model.fit(X, y, batch_size=batch_size, epochs=nb_epoch, verbose=verbose, callbacks=[checkpointer])
    model.save('GenerativeModel_compiled')
    
train_model(model, x, y, verbose=verbose)

Epoch 1/100
367/367 [==============================] - ETA: 0s - loss: 1.8716
Epoch 00001: loss improved from inf to 1.87155, saving model to weights_E.hdf5
367/367 [==============================] - 28s 76ms/step - loss: 1.8716
Epoch 2/100
367/367 [==============================] - ETA: 0s - loss: 1.8149
Epoch 00002: loss improved from 1.87155 to 1.81495, saving model to weights_E.hdf5
367/367 [==============================] - 28s 75ms/step - loss: 1.8149
Epoch 3/100
367/367 [==============================] - ETA: 0s - loss: 1.7658
Epoch 00003: loss improved from 1.81495 to 1.76580, saving model to weights_E.hdf5
367/367 [==============================] - 27s 75ms/step - loss: 1.7658
Epoch 4/100
367/367 [==============================] - ETA: 0s - loss: 1.7256
Epoch 00004: loss improved from 1.76580 to 1.72559, saving model to weights_E.hdf5
367/367 [==============================] - 28s 76ms/step - loss: 1.7256
Epoch 5/100
367/367 [==============================] - ETA: 0s - loss: 1

367/367 [==============================] - ETA: 0s - loss: 1.4168
Epoch 00037: loss improved from 1.42117 to 1.41682, saving model to weights_E.hdf5
367/367 [==============================] - 28s 78ms/step - loss: 1.4168
Epoch 38/100
367/367 [==============================] - ETA: 0s - loss: 1.4106
Epoch 00038: loss improved from 1.41682 to 1.41060, saving model to weights_E.hdf5
367/367 [==============================] - 28s 78ms/step - loss: 1.4106
Epoch 39/100
367/367 [==============================] - ETA: 0s - loss: 1.4030
Epoch 00039: loss improved from 1.41060 to 1.40304, saving model to weights_E.hdf5
367/367 [==============================] - 28s 77ms/step - loss: 1.4030
Epoch 40/100
367/367 [==============================] - ETA: 0s - loss: 1.3946
Epoch 00040: loss improved from 1.40304 to 1.39462, saving model to weights_E.hdf5
367/367 [==============================] - 29s 79ms/step - loss: 1.3946
Epoch 41/100
367/367 [==============================] - ETA: 0s - loss: 1.396

367/367 [==============================] - ETA: 0s - loss: 1.2815
Epoch 00073: loss did not improve from 1.27077
367/367 [==============================] - 26s 72ms/step - loss: 1.2815
Epoch 74/100
367/367 [==============================] - ETA: 0s - loss: 1.2693
Epoch 00074: loss improved from 1.27077 to 1.26928, saving model to weights_E.hdf5
367/367 [==============================] - 27s 73ms/step - loss: 1.2693
Epoch 75/100
367/367 [==============================] - ETA: 0s - loss: 1.2605
Epoch 00075: loss improved from 1.26928 to 1.26048, saving model to weights_E.hdf5
367/367 [==============================] - 27s 72ms/step - loss: 1.2605
Epoch 76/100
367/367 [==============================] - ETA: 0s - loss: 1.2632
Epoch 00076: loss did not improve from 1.26048
367/367 [==============================] - 26s 72ms/step - loss: 1.2632
Epoch 77/100
367/367 [==============================] - ETA: 0s - loss: 1.2421
Epoch 00077: loss improved from 1.26048 to 1.24211, saving model to we

In [ ]:
np.random.seed(1337)

In [ ]:
def sample(preds):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 0.2
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
N_CHARS = None

def create_index_char_map(corpus, verbose=0):
    chars = sorted(list(set(corpus)))
    global N_CHARS
    N_CHARS = len(chars)
    if verbose:
        print('No. of unique characters:', N_CHARS)
    char_to_idx = {c: i for i, c in enumerate(chars)}
    idx_to_char = {i: c for i, c in enumerate(chars)}
    return chars, char_to_idx, idx_to_char

chars, char_to_idx, idx_to_char = create_index_char_map(text, verbose=verbose)

No. of unique characters: 28


In [ ]:
def generate_tweets(model, corpus, char_to_idx, idx_to_char, n_tweets=10, verbose=0): 
    model.load_weights('weights_E.hdf5')
    tweets = []
    spaces_in_corpus = np.array([idx for idx in range(len(corpus)) if corpus[idx] == ' '])
    for i in range(1, n_tweets + 1):
        begin = np.random.choice(spaces_in_corpus)
        tweet = u''
        sequence = corpus[begin:begin + maxlen]
        tweet += sequence
        if verbose:
            print('Tweet no. %03d' % i)
            print('=' * 13)
            print('Generating with seed:')
            print(sequence)
            print('_' * len(sequence))
        for _ in range(100):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sequence):
                x[0, t, char_to_idx[char]] = 1.0

            preds = model.predict(x, verbose=0)[0]
            next_idx = sample(preds)
            next_char = idx_to_char[next_idx]

            tweet += next_char
            sequence = sequence[1:] + next_char
        if verbose:
            print(tweet)
            print()
        tweets.append(tweet)
    return tweets

tweets = generate_tweets(model, text, char_to_idx, idx_to_char, verbose=verbose)

Tweet no. 001
Generating with seed:
 enterprise
fred hollow inured hope gene
________________________________________


c:\users\mau\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 enterprise
fred hollow inured hope genefit claim state week adest wear mask best post service contact supplay said walk case study bussia c

Tweet no. 002
Generating with seed:
 karl
would wear mask advertise job like
________________________________________
 karl
would wear mask advertise job like carrer assist world first train senior student story person thank doman story policy state research

Tweet no. 003
Generating with seed:
 couture pm speak listen
lockdown restri
________________________________________
 couture pm speak listen
lockdown restrict world blook complete lockdown week adest plan go research surveyan study wear mask story world fi

Tweet no. 004
Generating with seed:
 mask advertise job like murder wear mas
________________________________________
 mask advertise job like murder wear mask story world fire complete lockdown moneth support continue study coronaviru response stay street f

Tweet no. 005
Generating with seed:
 mass billion made big company want mone
_

### Evaluating the model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(sentences)
Xval = vectorizer.transform(tweets)
print(pairwise_distances(Xval, Y=tfidf, metric='cosine').min(axis=1).mean())

0.4433820981217025
